In [1]:
import matplotlib.pyplot as plt
import numpy as np
import itertools

In [2]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [1]:
def plot_sign():
    x = np.linspace(-8, 8, 100)
    y = np.sign(x)
    plt.ylim(-1.1, 1.1)
    plt.plot(x, y)

    ax = plt.gca()
    ax.set_xlabel(r'${\bf w} \cdot {\bf x}$')
    ax.set_ylabel(r'$\hat y$')
    
    plt.title(r'Perceptron class label as a function of ${\bf w} \cdot {\bf x}$')

    plt.show()

In [5]:
def plot_logistic():
    x = np.linspace(-8, 8, 100)
    y = sigmoid(x)
    plt.ylim(-0.1, 1.1)
    plt.plot(x, y)
    plt.axvline(x = 0, ls = '--', lw = 0.5, color = 'black')
    plt.axhline(y = 0.5, ls = '--', lw = 0.5, color = 'black')
    ax = plt.gca()
    ax.set_xlabel(r'${\bf w} \cdot {\bf x}$')
    ax.set_ylabel(r'$p_{\bf w} ^{1}({\bf x})$')
    plt.show()

In [9]:
def plot_MLE():
    x = np.linspace(0.01, 0.99, 100)
    y = np.power(x, 7) * np.power(1 - x, 3)
    ax = plt.gca()
    ax.set_xlabel(r'$\theta$')
    ax.set_ylabel(r'$L(D; \theta)$')
    plt.title(r'Probability of 7 heads and 3 tails as a function of $p({\tt heads})$')
    plt.plot(x, y)
    plt.show()    

In [12]:
def plot_ln():
    x = np.linspace(0.01, 0.99, 100)
    y = np.log(x)
    ax = plt.gca()
    ax.set_xlabel(r'$p$')
    ax.set_ylabel(r'$\log(p)$')
    plt.title(r'The log is monotone with respect to its argument')
    plt.plot(x, y)
    plt.show()    

In [25]:
def plot_weights(weights):
    if not type(weights) == list:
        weights = list(weights)
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize = (10, 3))
    fig.tight_layout(pad = 2)
    
    ax1.set_title(r'$||w||$')
    ax1.plot(np.linalg.norm(weights, axis = 1))
    
    ax2.set_xlabel('Number of Instances Processed')
    ax2.set_title(r'$m$ (slope)')
    ax2.plot([-w[1]/w[2] for w in weights])
    
    ax3.set_title(r'$b$ (intercept)')
    ax3.plot([-w[0]/w[2] for w in weights])
    
    plt.show()

In [37]:
def plot_probabilities(df, weights, label = 'y'):
    
    features = list(df.columns)
    features.remove(label)

    p = df[features].apply(lambda row: sigmoid(np.dot(row, weights[1:]) + weights[0]), axis = 1).values
    p = p.tolist()
    p.sort()
    
    plt.ylim(-0.05, 1.05)
    plt.xlabel('')
    plt.xticks([])
    plt.ylabel(r'$p_{\bf w}^1({\bf x})$')
    plt.plot(p)
    plt.show()

In [2]:
def lr_classify(row, weights, labels):
    p = {}
    for label in labels:
        if label in weights:
            p[label] = np.exp(np.dot(row, weights[label]))
        else:
            p[label] = 1
    s = 1 + sum(p.values())
    for label in labels:
        p[label] /= s
    return max(p.items(), key=lambda x: x[1])

In [3]:
def plot_lr_regions(df, weights, label = 'y'):
    
    features = list(df.columns)
    features.remove(label)
    labels = sorted(df[label].unique())

    pts = []
    for feature in features:
        pts.append(np.arange(min(df[feature]), max(df[feature]), 0.1))
    pts = list(itertools.product(*pts))
    
    data = {'_x0_':np.full(len(pts), 1)}
    for idx, feature in enumerate(features):
        data[feature] = [pt[idx] for pt in pts]
    
    df2 = pd.DataFrame(data = data)
    preds = df2.apply(lambda row: lr_classify(row, weights, labels), axis = 1)
    df2[label] = [pred[0] for pred in preds]
    
    plot_2d(df2.drop('_x0_', axis = 1), 
            color_idx = [pred[1] for pred in preds], 
            weights = weights, show_separator = True)